In [ ]:
import numpy as np
import pandas as pd
import random
from isodisreg import idr
from statsmodels.nonparametric.bandwidths import bw_silverman
from helper_functions import llscore, crps_smooth, optimize_paras_onefit, optimize_paras_cvfit

In [ ]:
# Simulate data
n = 1000
forecast = np.random.uniform(low=0.0, high=10.0, size=n)
y_true = np.random.gamma(shape = np.sqrt(x), scale=np.minimum(np.maximum(x, 2), 8), size=n)  

In [ ]:
# Divide into train and test: 80 / 20
split_ix = int(np.floor(n * 0.8))
ix_data = np.arange(n)
random.shuffle(ix_data)
forecast_train = forecast[:split_ix]
forecast_test = forecast[split_ix:]
y_true_train = y_true[:split_ix]
y_true_test = y_true[split_ix:]

In [ ]:
# Fit EasyUQ
fitted_idr = idr(y_train, pd.DataFrame({"fore": forecast_train}, columns=["fore"]))
preds_test = fitted_idr.predict(pd.DataFrame({"fore": forecast_test}, columns=["fore"]))

In [ ]:
# Smoothing with ONE-FIT:
h, df = optimize_paras_onefit(fitted_idr, y_true_train)

ll = np.round(llscore(preds_test, y_true_test, h, df), 4)
crps_scdf = np.round(crps_smooth(preds_test, y_true_test, h, df), 4)
crps_idr = np.round(np.mean(preds_test.crps(y_true_test)), 4)

print('CRPS IDR: %d' % crps_idr)
print('CRPS smooth: %d' % crps_scdf)
print('Log score: %d' % ll)

In [ ]:
# Smoothing with cross validation:
h, df = optimize_paras_cvfit(fitted_idr, y_true_train)

ll = np.round(llscore(preds_test, y_true_test, h, df), 4)
crps_scdf = np.round(crps_smooth(preds_test, y_true_test, h, df), 4)
crps_idr = np.round(np.mean(preds_test.crps(y_true_test)), 4)

print('CRPS IDR: %d' % crps_idr)
print('CRPS smooth: %d' % crps_scdf)
print('Log score: %d' % ll)

In [ ]:
# Smoothing using Gaussian kernel and Silverman's rule of thumb:
rot = bw_silverman(y_true_train)

ll = np.round(llscore(preds_test, y_true_test, h = rot, df=None), 4)
crps_scdf = np.round(crps_smooth(preds_test, y_true_test, h = rot, df=None), 4)
crps_idr = np.round(np.mean(preds_test.crps(y_true_test)), 4)

print('CRPS IDR: %d' % crps_idr)
print('CRPS smooth: %d' % crps_scdf)
print('Log score: %d' % ll)